## Setup

In [150]:
from neo4j import GraphDatabase, Record


# Connect to DB
URI = 'neo4j://localhost:7687/'
AUTH = ('neo4j', '123neo4j')

driver = GraphDatabase.driver(URI, auth=AUTH)

driver.verify_connectivity()


# Remove old data
driver.execute_query('MATCH (a) -[r] -> () DELETE a, r')
driver.execute_query('MATCH (a) DELETE a')


# Import nodes
driver.execute_query(
    '''
    LOAD CSV WITH HEADERS FROM $url AS row
    MERGE (i:Item {itemID: toInteger(row.itemID), name: row.name, price: toInteger(row.price)});
    ''',
    url='https://raw.githubusercontent.com/Artem-Litkovskyi/kpi-db/master/lab5/import/items.csv'
)

driver.execute_query(
    '''
    LOAD CSV WITH HEADERS FROM $url AS row
    MERGE (c:Customer {customerID: toInteger(row.customerID), name: row.name});
    ''',
    url='https://raw.githubusercontent.com/Artem-Litkovskyi/kpi-db/master/lab5/import/customers.csv'
)

driver.execute_query(
    '''
    LOAD CSV WITH HEADERS FROM $url AS row
    MERGE (o:Order {orderID: toInteger(row.orderID)});
    ''',
    url='https://raw.githubusercontent.com/Artem-Litkovskyi/kpi-db/master/lab5/import/bought.csv'
)


# Import arrows
driver.execute_query(
    '''
    LOAD CSV WITH HEADERS FROM $url AS row
    MATCH (c:Customer {customerID: toInteger(row.customerID)})
    MATCH (o:Order {orderID: toInteger(row.orderID)})
    CREATE (c)-[:BOUGHT]->(o);
    ''',
    url='https://raw.githubusercontent.com/Artem-Litkovskyi/kpi-db/master/lab5/import/bought.csv'
)

driver.execute_query(
    '''
    LOAD CSV WITH HEADERS FROM $url AS row
    MATCH (o:Order {orderID: toInteger(row.orderID)})
    MATCH (i:Item {itemID: toInteger(row.itemID)})
    CREATE (o)-[:CONTAINS]->(i);
    ''',
    url='https://raw.githubusercontent.com/Artem-Litkovskyi/kpi-db/master/lab5/import/contains.csv'
)

driver.execute_query(
    '''
    LOAD CSV WITH HEADERS FROM $url AS row
    MATCH (c:Customer {customerID: toInteger(row.customerID)})
    MATCH (i:Item {itemID: toInteger(row.itemID)})
    CREATE (c)-[:VIEWED]->(i);
    ''',
    url='https://raw.githubusercontent.com/Artem-Litkovskyi/kpi-db/master/lab5/import/viewed.csv'
)

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x10fde8c80>, keys=[])

## Queries

In [151]:
def print_nodes(records, record_key, *keys):
    nodes = tuple(map(lambda r: r.data()[record_key], records))

    if len(keys) == 0:
        keys = nodes[0].keys()

    print_records(nodes, *keys)


def print_aggregate(records, *keys):
    for k in keys:
        print(f'{k.upper()}: {records[0].data()[k]}')


def print_records(records, *keys):
    values = {k: [] for k in keys}
    for r in records:
        if type(r) == Record:
            r = r.data()
        for k in keys:
            values[k].append(r[k])
    
    lenght = {}
    for k in keys:
        lenght[k] = max(map(lambda x: len(str(x)), values[k]))
    
    for i in range(len(records)):
        print(' '.join(f'{k.capitalize()}: {values[k][i]: <{lenght[k]}}' for k in keys))

In [152]:
order_id = 3

records, summary, keys = driver.execute_query(
    'MATCH (:Order {orderID: $order_id})-[:CONTAINS]->(i) RETURN i',
    order_id=order_id
)

print_nodes(records, 'i', 'name', 'price')

records, summary, keys = driver.execute_query(
    'MATCH (:Order {orderID: $order_id})-[:CONTAINS]->(i) RETURN sum(i.price) AS sum',
    order_id=order_id
)

print_aggregate(records, 'sum')

Name: Samsung Galaxy S25 Ultra Price: 67999
Name: Apple iPhone 16 Pro      Price: 61499
Name: Samsung UE43T5300AUXUA   Price: 15199
SUM: 144697


In [153]:
customer_id = 1

records, summary, keys = driver.execute_query(
    'MATCH (:Customer {customerID: $customer_id})-[:BOUGHT]->()-[:CONTAINS]->(i) RETURN i',
    customer_id=customer_id
)

print('BOUGHT:')
print_nodes(records, 'i', 'name', 'price')

records, summary, keys = driver.execute_query(
    '''
    MATCH (:Customer {customerID: $customer_id})-[:BOUGHT]->()-[:CONTAINS]->(i)
    RETURN count(i) AS count, sum(i.price) AS sum
    ''',
    customer_id=customer_id
)

print_aggregate(records, 'count', 'sum')

records, summary, keys = driver.execute_query(
    'MATCH (:Customer {customerID: $customer_id})-[:VIEWED]->(i) RETURN i',
    customer_id=customer_id
)

print('\nVIEWED:')
print_nodes(records, 'i', 'name', 'price')

BOUGHT:
Name: Apple iPhone 13      Price: 25499
Name: Philips 43PUS8009/12 Price: 20999
COUNT: 2
SUM: 46498

VIEWED:
Name: Apple Watch SE       Price: 12599
Name: Apple iPhone 13      Price: 25499
Name: Philips 43PUS8009/12 Price: 20999


In [154]:
records, summary, keys = driver.execute_query(
    '''
    MATCH (i:Item)<-[:CONTAINS]-()
    RETURN i.name AS name, count(i) AS count
    ORDER BY count DESC
    ''',
    customer_id=customer_id
)

print_records(records, *keys)

Name: Apple iPhone 13            Count: 4
Name: Apple Watch SE             Count: 4
Name: Xiaomi Redmi Watch 5       Count: 4
Name: Apple iPhone 16 Pro        Count: 3
Name: Samsung Galaxy S25 Ultra   Count: 3
Name: Samsung Galaxy Watch Ultra Count: 3
Name: Philips 43PUS8009/12       Count: 3
Name: LG 50UR78006LK             Count: 2
Name: Samsung Galaxy A16         Count: 1
Name: Samsung UE43T5300AUXUA     Count: 1


In [155]:
item_id = 3

records, summary, keys = driver.execute_query(
    'MATCH (i:Item {itemID: $item_id}) RETURN i',
    item_id=item_id
)

print('ITEM:')
print_nodes(records, 'i', 'name', 'price')

records, summary, keys = driver.execute_query(
    'MATCH (:Item {itemID: $item_id})<-[:CONTAINS]-()-[:CONTAINS]->(i) RETURN i',
    item_id=item_id
)

print('\nBOUGHT WITH:')
print_nodes(records, 'i', 'name', 'price')

records, summary, keys = driver.execute_query(
    'MATCH (:Item {itemID: $item_id})<-[:CONTAINS]-()<-[:BOUGHT]->(c) RETURN c',
    item_id=item_id
)

print('\nORDERED BY:')
print_nodes(records, 'c', 'name')

ITEM:
Name: Samsung Galaxy S25 Ultra Price: 67999

BOUGHT WITH:
Name: Samsung Galaxy Watch Ultra Price: 28999
Name: Xiaomi Redmi Watch 5       Price: 4499 
Name: Apple Watch SE             Price: 12599
Name: Apple iPhone 16 Pro        Price: 61499
Name: Samsung UE43T5300AUXUA     Price: 15199
Name: LG 50UR78006LK             Price: 21499
Name: Apple Watch SE             Price: 12599

ORDERED BY:
Name: Diana Wilson
Name: Ian Wilson  
Name: Fiona Garcia


In [156]:
customer_id = 2

records, summary, keys = driver.execute_query(
    'MATCH (:Customer {customerID: $customer_id})-[:VIEWED]->(i) RETURN i',
    customer_id=customer_id
)

print('\nVIEWED:')
print_nodes(records, 'i', 'name', 'price')

records, summary, keys = driver.execute_query(
    'MATCH (:Customer {customerID: $customer_id})-[:BOUGHT]->()-[:CONTAINS]->(i) RETURN i',
    customer_id=customer_id
)

print('\nBOUGHT:')
print_nodes(records, 'i', 'name', 'price')

records, summary, keys = driver.execute_query(
    '''
    MATCH (c:Customer {customerID: $customer_id})-[:VIEWED]->(i)
    WHERE NOT (c)-[:BOUGHT]->()-[:CONTAINS]->(i)
    RETURN i
    ''',
    customer_id=customer_id
)

print('\nVIEWED BUT HAVEN\'T BOUGHT:')
print_nodes(records, 'i', 'name', 'price')


VIEWED:
Name: Samsung Galaxy A16         Price: 9499 
Name: LG 50UR78006LK             Price: 21499
Name: Samsung Galaxy S25 Ultra   Price: 67999
Name: Apple iPhone 16 Pro        Price: 61499
Name: Apple Watch SE             Price: 12599
Name: Philips 43PUS8009/12       Price: 20999
Name: Samsung Galaxy Watch Ultra Price: 28999
Name: Apple iPhone 13            Price: 25499

BOUGHT:
Name: Apple Watch SE       Price: 12599
Name: Apple iPhone 13      Price: 25499
Name: Philips 43PUS8009/12 Price: 20999
Name: LG 50UR78006LK       Price: 21499

VIEWED BUT HAVEN'T BOUGHT:
Name: Samsung Galaxy A16         Price: 9499 
Name: Samsung Galaxy S25 Ultra   Price: 67999
Name: Apple iPhone 16 Pro        Price: 61499
Name: Samsung Galaxy Watch Ultra Price: 28999
